<a href="https://colab.research.google.com/github/LinCheungS/ml_sklearn_tutorial/blob/master/%E7%BA%BF%E6%80%A7%E5%9B%9E%E5%BD%92%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 线性回归算法

## 原理

### 步骤

- 根据公式求出系数
- 比较系数预测的结果和真实的结果

### 解释
   对于一些数据存在这么一条线,最大程度的拟合所有数据  
   目标找到a,b,使得  $ \sum_{i=1}^{m}(y^{(i)} - ax^{(i)}- b)^2 $ 的值尽可能的小  
   这个值通常叫做损失函数(loss function)

### 基本公式
### 最小二乘法(Least squares)

$$ a = \dfrac{\sum_{i=1}^{m}(x^{(i)} - \hat{x})(y^{(i)} - \hat{y})}{\sum_{i=1}^{m}(x^{(i)} - \hat{x})^2} $$  
$$ b = \hat{y} - a\hat{x}$$   
<center> note:使用向量化加速运算 </center>

### 多元线性回归公式

应该怎么从一大堆数据里求出回归方程呢？假定输入数据存放在矩阵X中，结果存放在向量y中:  
$$
X=\left[\begin{array}{lll}
x_{11} & x_{12} & x_{13} \\
x_{21} & x_{22} & x_{23} \\
x_{31} & x_{32} & x_{33}
\end{array}\right], y=\left[\begin{array}{l}
y_{1} \\
y_{2} \\
y_{3}
\end{array}\right]
$$

而回归系数存放在向量w中：  
$$
\omega=\left[\begin{array}{l}
\omega_{1} \\
\omega_{2} \\
\omega_{3}
\end{array}\right]
$$
那么对于给定的数据x1，即矩阵X的第一列数据，预测结果u1将会通过如下公式给出：  
$$
u_{1}=\left[\begin{array}{l}
x_{11} \\
x_{21} \\
x_{31}
\end{array}\right]^{T} *\left[\begin{array}{l}
\omega_{1} \\
\omega_{2} \\
\omega_{3}
\end{array}\right]
$$
平方误差和可以写做:  
$$
\sum_{i=1}^{m}\left(y_{i}-x_{i}^{T} \omega\right)^{2}
$$
用矩阵表示还可以写做：
$$
(y-X \omega)^{T}(y-X \omega)
$$

# Sklearn实现

## 自我实现(最小二乘)

In [1]:
import numpy as np

class SimpleLinearRegression:
    def __init__(self):
        """初始化Simple Linear Regression模型"""
        self.a_ = None
        self.b_ = None

    def fit(self, x_train, y_train):
        """根据训练数据集x_train, y_train训练Simple Linear Regression模型"""
        assert x_train.ndim == 1, \
            "Simple Linear Regressor can only solve single feature training data."
        assert len(x_train) == len(y_train), \
            "the size of x_train must be equal to the size of y_train"

        x_mean = np.mean(x_train)
        y_mean = np.mean(y_train)

        #向量化运算
        self.a_ = (x_train - x_mean).dot(y_train - y_mean) / (
            x_train - x_mean).dot(x_train - x_mean)
        self.b_ = y_mean - self.a_ * x_mean

        return self

    def predict(self, x_predict):
        """给定待预测数据集x_predict，返回表示x_predict的结果向量"""
        assert x_predict.ndim == 1, \
            "Simple Linear Regressor can only solve single feature training data."
        assert self.a_ is not None and self.b_ is not None, \
            "must fit before predict!"

        return np.array([self._predict(x) for x in x_predict])

    def _predict(self, x_single):
        """给定单个待预测数据x，返回x的预测结果值"""
        return self.a_ * x_single + self.b_

    def __repr__(self):
        return "SimpleLinearRegression()"

## 自我实现多元线性回归

In [2]:
import numpy as np
from sklearn.metrics import r2_score

class LinearRegression:

    def __init__(self):
        """初始化Linear Regression模型"""
        self.coef_ = None
        self.intercept_ = None
        self._theta = None

    def fit_normal(self, X_train, y_train):
        """根据训练数据集X_train, y_train训练Linear Regression模型"""
        assert X_train.shape[0] == y_train.shape[0], \
            "the size of X_train must be equal to the size of y_train"

        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        self._theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)

        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]

        return self

    def predict(self, X_predict):
        """给定待预测数据集X_predict，返回表示X_predict的结果向量"""
        assert self.intercept_ is not None and self.coef_ is not None, \
            "must fit before predict!"
        assert X_predict.shape[1] == len(self.coef_), \
            "the feature number of X_predict must be equal to X_train"

        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        return X_b.dot(self._theta)

    def score(self, X_test, y_test):
        """根据测试数据集 X_test 和 y_test 确定当前模型的准确度"""

        y_predict = self.predict(X_test)
        return r2_score(y_test, y_predict)

    def __repr__(self):
        return "LinearRegression()"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

boston = datasets.load_boston()
x = boston.data
y = boston.target
# 预先处理
x = x[y < 50]
y = y[y < 50]

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=666)
reg = LinearRegression()
reg.fit_normal(X_train,y_train)
reg.score(X_test,y_test)

0.8129794056212925

## 使用sklearn多元线性回归

In [4]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)
lin_reg.score(X_test,y_test)

0.8129794056212811